In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
run = wandb.init(
    project='Fine-tune Gemma-2-2b-it on HealthCare Dataset', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aidenyang66 (yyfsss). Use `wandb login --relogin` to force relogin


In [3]:
base_model = "google/gemma-2-2b-it"
dataset_name = "yunfan-y/trump-qa"
new_model = "yunfan-y/Gemma-2-2b-it-trump-500"

In [4]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [5]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

In [6]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
model = get_peft_model(model, peft_config)

#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["input"]},
               {"role": "assistant", "content": row["output"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
)

dataset
dataset['text'][3]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 11092.17 examples/s]


'<bos><start_of_turn>user\nWhen are you planning to announce your candidacy for the 2016 Presidential election?<end_of_turn>\n<start_of_turn>model\nwhen are you going to announce your bid for 2016 Thanks for your nice thoughts our Country is a mess<end_of_turn>\n'

In [7]:
dataset = dataset.train_test_split(test_size=0.1)

In [8]:
# Setting Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=30,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length= 256,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

model.config.use_cache = False
trainer.train()

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 100/100 [00:00<00:00, 8544.98 examples/s]
c:\Users

{'loss': 5.7641, 'grad_norm': 7.3076395988464355, 'learning_rate': 6.666666666666667e-06, 'epoch': 0.0}


  1%|          | 2/225 [00:02<03:55,  1.05s/it]

{'loss': 5.5338, 'grad_norm': 7.342902660369873, 'learning_rate': 1.3333333333333333e-05, 'epoch': 0.01}


  1%|▏         | 3/225 [00:02<03:08,  1.18it/s]

...

                                                 
100%|██████████| 225/225 [02:08<00:00,  2.33it/s]

{'eval_loss': 1.8984450101852417, 'eval_runtime': 4.5482, 'eval_samples_per_second': 21.987, 'eval_steps_per_second': 10.993, 'epoch': 1.0}


100%|██████████| 225/225 [02:10<00:00,  1.73it/s]

{'train_runtime': 130.329, 'train_samples_per_second': 6.906, 'train_steps_per_second': 1.726, 'train_loss': 2.3128874338997734, 'epoch': 1.0}


TrainOutput(global_step=225, training_loss=2.3128874338997734, metrics={'train_runtime': 130.329, 'train_samples_per_second': 6.906, 'train_steps_per_second': 1.726, 'total_flos': 496856899252224.0, 'train_loss': 2.3128874338997734, 'epoch': 1.0})

# prepare the complete model

In [14]:
""" base_model_url = "google/gemma-2-2b-it"
new_model_url = "yunfan-y/Gemma-2-2b-it-trump-1000-adapter-quantized" """

In [15]:
""" from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch


# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cpu",
) """

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.46it/s]


In [16]:
""" model = PeftModel.from_pretrained(base_model_reload, new_model_url)
model = model.merge_and_unload() """

In [18]:
""" model.push_to_hub("yunfan-y/Gemma-2-2b-it-trump-1000-complete-quantized")
tokenizer.push_to_hub("yunfan-y/Gemma-2-2b-it-trump-1000-complete-quantized") """

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 1.67M/4.99G [00:00<09:17, 8.95MB/s] 
model-00001-of-00002.safetensors:   0%|          | 5.03M/4.99G [00:00<04:14, 19.6MB/s]
model-00001-of-00002.safetensors:   0%|          | 9.96M/4.99G [00:00<02:58, 27.9MB/s]
model-00001-of-00002.safetensors:   0%|          | 14.1M/4.99G [00:00<02:38, 31.4MB/s]

model-00001-of-00002.safetensors:   1%|          | 30.9M/4.99G [00:00<01:51, 44.5MB/s]

model-00001-of-00002.safetensors:   1%|          | 47.5M/4.99G [00:01<01:56, 42.3MB/s]

model-00001-of-00002.safetensors:   1%|          | 57.7M/4.99G [00:01<02:19, 35.4MB/s]

model-00001-of-00002.safetensors:   2%|▏         | 79.0M/4.99G [00:02<01:46, 45.9MB/s]

model-00001-of-00002.safetensors:   2%|▏         | 97.0M/4.99G [00:02<02:44, 29.7MB/s]
model-00001-of-00002.safetensors:   2%|▏         | 103M/4.99G [00:03<02:15, 36.1MB/s] 
model-00001-of-00002.safetensors:   2%|▏     

CommitInfo(commit_url='https://huggingface.co/yunfan-y/Gemma-2-2b-it-trump-1000-complete-quantized/commit/707d79d05171772806489cb10163a29e006158dc', commit_message='Upload tokenizer', commit_description='', oid='707d79d05171772806489cb10163a29e006158dc', pr_url=None, pr_revision=None, pr_num=None)